# SkillUp IA – Recomendação de Carreira com Gemini

Este notebook demonstra o uso de **IA Generativa (Deep Learning)** via **Google Gemini**
para gerar recomendações de carreira, habilidades e cursos com base no perfil do usuário.

## Instruções
1. Gere uma API key gratuita em: https://aistudio.google.com/app/apikey
2. Copie a chave (começa com `AIza...`).
3. Substitua `SUA_CHAVE_GEMINI_AQUI` na célula abaixo pela sua chave real.
4. Execute as células em ordem.


In [ ]:
!pip install -q google-generativeai requests

import json
import google.generativeai as genai
import requests

# ==========================
# 1. CONFIGURAÇÃO DA API KEY (GEMINI)
# ==========================

# Troque aqui pela SUA chave do Gemini (pega em https://aistudio.google.com/app/apikey)
GEMINI_API_KEY = "CHAVE_GEMINI"

if GEMINI_API_KEY == "CHAVE_GEMINI":
    raise ValueError("⚠️ Defina a variável GEMINI_API_KEY com a sua chave do Gemini antes de continuar.")

genai.configure(api_key=GEMINI_API_KEY)

# Modelo usado (rápido e bom para texto estruturado)
model = genai.GenerativeModel("gemini-2.0-flash")

# ==========================
# 2. FUNÇÃO DE IA (DEEP LEARNING)
# ==========================

def gerar_recomendacao_carreira(objetivo: str, habilidades: list[str], experiencia: str) -> dict:
    """
    Usa o modelo Gemini para gerar recomendações de carreira, habilidades e cursos
    com base no perfil do usuário.
    Retorna um dicionário Python (que pode ser convertido para JSON).
    """

    system_prompt = """
    Você é uma IA especialista em futuro do trabalho, carreiras emergentes,
    requalificação profissional (reskilling/upskilling) e orientação de estudos.

    Sua tarefa é analisar o perfil do usuário (objetivo, habilidades, experiência)
    e devolver um JSON estruturado com:

    {
      "perfilAnalise": "...",
      "profissoesRecomendadas": ["..."],
      "habilidadesParaDesenvolver": {
        "hardSkills": ["..."],
        "softSkills": ["..."]
      },
      "cursosRecomendados": [
        { "nome": "...", "descricao": "...", "motivo": "..." }
      ]
    }

    IMPORTANTE:
    - Responda SEMPRE em JSON válido.
    - NÃO inclua nenhum texto fora do JSON.
    - Fale em português.
    """

    user_prompt = {
        "objetivo": objetivo,
        "habilidades": habilidades,
        "experiencia": experiencia
    }

    full_prompt = (
        system_prompt
        + "\n\nA seguir estão os dados do usuário em JSON. "
        + "Analise e devolva APENAS um JSON com as recomendações:\n\n"
        + json.dumps(user_prompt, ensure_ascii=False, indent=2)
    )

    response = model.generate_content(full_prompt)
    raw_text = response.text.strip()

    # ==========================
    # LIMPEZA DOS ```json ... ```
    # ==========================
    if raw_text.startswith("```"):
        # remove crases do começo/fim
        raw_text = raw_text.strip("`").strip()
        # se começar com "json", remove essa palavra
        if raw_text.lower().startswith("json"):
            raw_text = raw_text[4:].lstrip()

    # ==========================
    # TENTAR CONVERTER PARA JSON
    # ==========================
    try:
        data = json.loads(raw_text)
    except json.JSONDecodeError:
        # fallback simples: embrulha em um dict com erro
        data = {
            "erro": "Falha ao interpretar o JSON da IA.",
            "respostaBruta": raw_text
        }

    return data

print("✅ Função gerar_recomendacao_carreira pronta para uso com Gemini.")


In [ ]:
# ==========================
# 3. EXEMPLO DE USO / TESTE DA IA
# ==========================

objetivo_teste = "Quero migrar para a área de ciência de dados ou análise de dados."
habilidades_teste = ["Excel", "Lógica de Programação", "SQL básico", "Análise de negócios"]
experiencia_teste = "3 anos trabalhando como analista administrativo em uma empresa de logística."

resultado = gerar_recomendacao_carreira(
    objetivo=objetivo_teste,
    habilidades=habilidades_teste,
    experiencia=experiencia_teste
)

print("===== RECOMENDAÇÃO DE CARREIRA (JSON GERADO PELA IA) =====")
print(json.dumps(resultado, ensure_ascii=False, indent=2))


In [ ]:
# ================================
# 4. SIMULAÇÃO DE INTEGRAÇÃO COM API .NET
# ================================

# Exemplo de payload que seria enviado para a API SkillUp em .NET
payload_para_api = {
    "usuarioId": 123,  # ID fictício do usuário no sistema SkillUp
    "objetivo": objetivo_teste,
    "habilidades": habilidades_teste,
    "experiencia": experiencia_teste,
    "recomendacaoIa": resultado  # JSON completo retornado pela IA
}

print("\n===== PAYLOAD PARA API .NET (EXEMPLO) =====")
print(json.dumps(payload_para_api, ensure_ascii=False, indent=2))

# ============================================
# 4.1 Enviando o payload para um endpoint MOCK
# ============================================
# Aqui usamos https://httpbin.org/post como se fosse a API .NET.
# Ele apenas devolve o que recebeu, servindo como simulação de integração HTTP.

url_mock_api = "https://httpbin.org/post"

response = requests.post(url_mock_api, json=payload_para_api)

print("\n===== RESPOSTA DO ENDPOINT MOCK (HTTPBIN) =====")
print("Status code:", response.status_code)

# Mostra só a parte 'json' que o httpbin devolveu (que é exatamente o que a gente mandou)
try:
    resp_json = response.json()
    print("\nJSON recebido pelo 'servidor' (simulando a API .NET):")
    print(json.dumps(resp_json.get("json", {}), ensure_ascii=False, indent=2))
except Exception as e:
    print("Não foi possível interpretar a resposta como JSON:", e)

print("""
🔗 Explicação:
- A IA (Gemini) gera o JSON de recomendação (resultado).
- Montamos um payload compatível com uma API REST .NET (SkillUp).
- Enviamos esse payload via HTTP POST para um endpoint (mock) que simula a API .NET.
- Em um cenário real, a URL seria algo como:
  http://localhost:5000/api/v1/recomendacoes

Assim, fica demonstrada a integração entre a solução de IA (disciplina de IoT/IA)
e uma API de backend em .NET (outra disciplina).
""")
